<a href="https://colab.research.google.com/github/panchamdesai777/Deep-Learning/blob/master/Intro%20to%20Deeplearning/Deep_Learning_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Problem statement:**


In this project we are working on the Lending club dataset. Lending Club is a peer to peer lending company based in the United States, in which investors provide funds for potential borrowers and investors earn a profit depending on the risk they take (the borrowers credit score).

From the given set of data we want to predict loan_status of the borrower. We have to predict the loan staus based on the features like Loan amount,payment plan,grade,verification status,recoveries etc. The loan status having the various categories like Fully paid,charged off,late,Issued,In a grace period etc.

**About the dataset**

A zipped file containing following items is given:

* train.csv:
The data file train.csv contains the 621165 loan status entries with the 80 features including the target feature.

* test.csv:
The datafile test.csv contains the 266214 loan status entries with the 79 features excluding the target feature.

* sample_submission.csv: Explained under the submission sub-heading

* LCDataDictionary.csv: The file contains data dictionary(Dictionary explaining what each feature of the dataset means) of the lending club dataset

* challenge_entity_student_template.ipynb: A template notebook explaining the task breakdown to solve the given problem statement (Learners are recommended to use it)

**Submission**

After training of the model on train.csv data, learner has to predict the target feature of the test.csv data using the trained model. The learner has to then submit a csv file with the predicted feature.

Sample submission file(sample_submission.csv) is given to you as reference to the format expected when you submit

Evaluation metrics
For this particular dataset we are using Accuracy as a evaluation metric.

Submissions will be evaluated based on Accuracy score

**Outcomes**

After completing this project, you will have the better understanding of how to build a basic neural network model. In this project, you will apply the following concepts.

* Neural Network Model creation
* Neural Network Model training
* Optimizing The Neural Network

In [110]:
#Importing Required Libraries
#_______________________________________________________________________________________________________________
import numpy as np
import gc
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style # for styling the graphs
# style.available (to know the available list of styles)
style.use('ggplot') # chosen style
plt.rc('xtick',labelsize=13) # to globally set the tick size
plt.rc('ytick',labelsize=13) # to globally set the tick size

# To print multiple outputs together
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Change column display number during print
pd.set_option('display.max_columns', 500)
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')
# To display float with 2 decimal, avoid scientific printing
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [111]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [112]:
#train data
path='/content/gdrive/My Drive/Intro_to_deep_learning/Project-1/train.csv'
train=pd.read_csv(path)
train

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,application_type_0,application_type_1,emp_length_0,emp_length_1,emp_length_2,emp_length_3,emp_length_4,emp_length_5,emp_length_6,emp_length_7,emp_length_8,emp_length_9,emp_length_10,emp_length_11,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,grade_6,home_ownership_0,home_ownership_1,home_ownership_2,home_ownership_3,home_ownership_4,home_ownership_5,initial_list_status_0,initial_list_status_1,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,purpose_11,purpose_12,purpose_13,pymnt_plan_0,pymnt_plan_1,term_0,term_1,verification_status_0,verification_status_1,verification_status_2,loan_status
0,20000.00,20000.00,20000.00,8.18,407.26,200000.00,8.76,0.00,1.00,14.00,0.00,47274.00,22.30,29.00,18626.77,18626.77,2027.21,2027.21,1373.23,653.98,0.00,0.00,0.00,407.26,0.00,1.00,0.00,0.00,304951.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1
1,7500.00,7500.00,7500.00,11.99,249.08,157000.00,19.04,0.00,1.00,8.00,0.00,20482.00,98.50,27.00,4897.17,4897.17,3485.51,3485.51,2602.83,882.68,0.00,0.00,0.00,252.47,0.00,1.00,0.00,0.00,543005.00,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1
2,11200.00,11200.00,11100.00,14.65,264.40,80000.00,16.74,1.00,2.00,12.00,0.00,6731.00,38.20,20.00,10410.24,10317.29,1568.17,1554.17,789.76,778.41,0.00,0.00,0.00,264.40,0.00,1.00,0.00,421.00,105430.00,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1
3,6000.00,6000.00,6000.00,11.99,199.26,55000.00,27.93,1.00,1.00,13.00,0.00,9240.00,48.60,20.00,2068.11,2068.11,4981.50,4981.50,3931.89,1049.61,0.00,0.00,0.00,199.26,0.00,1.00,0.00,406.00,56932.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1
4,10000.00,10000.00,10000.00,8.67,316.47,40000.00,16.53,0.00,0.00,9.00,0.00,10541.00,53.00,16.00,0.00,0.00,10545.54,10545.54,10000.00,545.54,0.00,0.00,0.00,5260.89,0.00,1.00,0.00,0.00,24462.00,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621160,12000.00,12000.00,12000.00,7.26,371.96,67000.00,14.22,0.00,1.00,6.00,0.00,15078.00,83.80,8.00,10176.45,10176.45,2222.08,2222.08,1823.55,398.53,0.00,0.00,0.00,371.96,0.00,1.00,0.00,0.00,21814.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1
621161,25000.00,25000.00,25000.00,13.35,573.32,130000.00,18.14,0.00,0.00,16.00,0.00,21661.00,65.40,30.00,20210.06,20210.06,8599.80,8599.80,4789.94,3809.86,0.00,0.00,0.00,573.32,0.00,1.00,0.00,0.00,356898.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1
621162,11200.00,11200.00,11200.00,16.55,396.81,51000.00,19.36,0.00,0.00,13.00,0.00,7439.00,31.40,18.00,10711.97,10711.97,773.02,773.02,488.03,284.99,0.00,0.00,0.00,396.81,0.00,1.00,0.00,0.00,22246.00,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1
621163,25725.00,25725.00,25725.00,18.24,656.61,75000.00,25.28,0.00,1.00,13.00,0.00,4827.00,17.10,31.00,19925.22,19925.22,12475.59,12475.59,5799.78,6675.81,0.00,0.00,0.00,656.61,0.00,1.00,0.00,0.00,310566.00,1,0,0,0,0,0,0,0,0,0,1

In [113]:
#test data
path='/content/gdrive/My Drive/Intro_to_deep_learning/Project-1/test.csv'
test=pd.read_csv(path)
test

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,application_type_0,application_type_1,emp_length_0,emp_length_1,emp_length_2,emp_length_3,emp_length_4,emp_length_5,emp_length_6,emp_length_7,emp_length_8,emp_length_9,emp_length_10,emp_length_11,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,grade_6,home_ownership_0,home_ownership_1,home_ownership_2,home_ownership_3,home_ownership_4,home_ownership_5,initial_list_status_0,initial_list_status_1,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,purpose_11,purpose_12,purpose_13,pymnt_plan_0,pymnt_plan_1,term_0,term_1,verification_status_0,verification_status_1,verification_status_2
0,12400.00,12400.00,12400.00,13.05,418.11,80460.00,10.17,0.00,0.00,12.00,0.00,14571.00,84.20,26.00,0.00,0.00,14647.02,14647.02,12400.00,2247.02,0.00,0.00,0.00,5869.16,0.00,1.00,0.00,0.00,493360.00,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
1,8000.00,8000.00,8000.00,10.99,261.88,265000.00,2.28,0.00,0.00,2.00,0.00,24553.00,98.20,10.00,4105.06,4105.06,4975.72,4975.72,3894.94,1080.78,0.00,0.00,0.00,261.88,0.00,1.00,0.00,0.00,927890.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
2,15000.00,15000.00,15000.00,6.24,457.97,110000.00,8.26,0.00,0.00,13.00,0.00,74016.00,39.70,39.00,13854.15,13854.15,1368.71,1368.71,1145.85,222.86,0.00,0.00,0.00,457.97,0.00,1.00,0.00,0.00,532715.00,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
3,8500.00,8500.00,8500.00,17.77,306.32,54000.00,5.31,1.00,3.00,14.00,0.00,6553.00,44.30,30.00,0.00,0.00,9323.40,9323.40,8500.00,823.40,0.00,0.00,0.00,7486.22,0.00,1.00,0.00,0.00,13793.00,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0
4,17600.00,17600.00,17600.00,12.29,587.02,60000.00,18.40,0.00,0.00,13.00,0.00,11013.00,51.20,19.00,16367.16,16367.16,1737.03,1737.03,1232.84,504.19,0.00,0.00,0.00,587.02,0.00,1.00,0.00,0.00,30674.00,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266209,16000.00,16000.00,16000.00,11.71,353.58,44000.00,13.70,0.00,0.00,7.00,0.00,23467.00,54.40,26.00,0.00,0.00,18001.22,18001.22,16000.00,2001.22,0.00,0.00,0.00,13412.52,0.00,1.00,0.00,225.70,139458.19,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
266210,9300.00,9300.00,9300.00,6.49,285.00,32000.00,26.59,1.00,1.00,17.00,1.00,6617.00,24.40,35.00,6647.31,6647.31,3128.29,3128.29,2652.69,475.60,0.00,0.00,0.00,285.00,0.00,1.00,0.00,0.00,36489.00,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
266211,35000.00,35000.00,34975.00,19.19,1286.33,85000.00,9.39,0.00,2.00,4.00,0.00,22407.00,68.30,14.00,32785.09,32761.67,3821.68,3818.95,2214.91,1606.77,0.00,0.00,0.00,1286.33,0.00,1.00,0.00,0.00,36156.00,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
266212,24000.00,24000.00,24000.00,11.14,787.33,57600.00,18.79,0.00,1.00,23.00,0.00,12549.00,49.40,50.00,0.00,0.00,27558.21,27558.21,24000.00,3558.21,0.00,0.00,0.00,11812.01,0.00,1.00,0.00,0.00,309860.00,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0

In [114]:
#target data
train['loan_status'].value_counts()

1    421245
5    145406
0     31673
9      8114
7      5922
6      4377
8      1650
4      1392
2       853
3       533
Name: loan_status, dtype: int64

In [115]:
#Drop the features havinng 100 % similar information
train.drop(['funded_amnt', 'funded_amnt_inv','out_prncp_inv','total_pymnt_inv','policy_code'],1,inplace=True)
test.drop(['funded_amnt', 'funded_amnt_inv','out_prncp_inv','total_pymnt_inv','policy_code'],1,inplace=True)


In [116]:
train.fillna(0,inplace=True)

In [117]:
test.fillna(0,inplace=True)

## **Removing Features with Low Variance**

---



In [118]:
from sklearn.feature_selection import VarianceThreshold
threshold_n=0.95
sel = VarianceThreshold(threshold=(threshold_n* (1 - threshold_n) ))
sel_var=sel.fit_transform(train)
train=train[train.columns[sel.get_support(indices=True)]] 
train

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,tot_coll_amt,tot_cur_bal,emp_length_0,emp_length_1,emp_length_2,emp_length_3,emp_length_4,emp_length_5,emp_length_7,emp_length_10,emp_length_11,grade_0,grade_1,grade_2,grade_3,grade_4,home_ownership_1,home_ownership_4,home_ownership_5,initial_list_status_0,initial_list_status_1,purpose_1,purpose_2,purpose_4,term_0,term_1,verification_status_0,verification_status_1,verification_status_2,loan_status
0,20000.00,8.18,407.26,200000.00,8.76,0.00,1.00,14.00,0.00,47274.00,22.30,29.00,18626.77,2027.21,1373.23,653.98,0.00,0.00,0.00,407.26,0.00,304951.00,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1
1,7500.00,11.99,249.08,157000.00,19.04,0.00,1.00,8.00,0.00,20482.00,98.50,27.00,4897.17,3485.51,2602.83,882.68,0.00,0.00,0.00,252.47,0.00,543005.00,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1
2,11200.00,14.65,264.40,80000.00,16.74,1.00,2.00,12.00,0.00,6731.00,38.20,20.00,10410.24,1568.17,789.76,778.41,0.00,0.00,0.00,264.40,421.00,105430.00,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0,1
3,6000.00,11.99,199.26,55000.00,27.93,1.00,1.00,13.00,0.00,9240.00,48.60,20.00,2068.11,4981.50,3931.89,1049.61,0.00,0.00,0.00,199.26,406.00,56932.00,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,1
4,10000.00,8.67,316.47,40000.00,16.53,0.00,0.00,9.00,0.00,10541.00,53.00,16.00,0.00,10545.54,10000.00,545.54,0.00,0.00,0.00,5260.89,0.00,24462.00,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621160,12000.00,7.26,371.96,67000.00,14.22,0.00,1.00,6.00,0.00,15078.00,83.80,8.00,10176.45,2222.08,1823.55,398.53,0.00,0.00,0.00,371.96,0.00,21814.00,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1
621161,25000.00,13.35,573.32,130000.00,18.14,0.00,0.00,16.00,0.00,21661.00,65.40,30.00,20210.06,8599.80,4789.94,3809.86,0.00,0.00,0.00,573.32,0.00,356898.00,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,1
621162,11200.00,16.55,396.81,51000.00,19.36,0.00,0.00,13.00,0.00,7439.00,31.40,18.00,10711.97,773.02,488.03,284.99,0.00,0.00,0.00,396.81,0.00,22246.00,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1
621163,25725.00,18.24,656.61,75000.00,25.28,0.00,1.00,13.00,0.00,4827.00,17.10,31.00,19925.22,12475.59,5799.78,6675.81,0.00,0.00,0.00,656.61,0.00,310566.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1


## Why it is necessary to scale features before feeding to neural network ?

There are 2 Reasons why we have to Normalize Input Features before Feeding them to Neural Network:

1. **Reason 1**: If a Feature in the Dataset is big in scale compared to others then this big scaled feature becomes dominating and as a result of that, Predictions of the Neural Network will not be Accurate.

 **Example:** In case of Employee Data, if we consider Age and Salary, Age will be a Two Digit Number while Salary can be 7 or 8 Digit (1 Million, etc..). In that Case, Salary will Dominate the Prediction of the Neural Network. But if we Normalize those Features, Values of both the Features will lie in the Range from (0 to 1).

2. **Reason 2**: Front Propagation of Neural Networks involves the Dot Product of Weights with Input Features. So, if the Values are very high (for Image and Non-Image Data), Calculation of Output takes a lot of Computation Time as well as Memory. Same is the case during Back Propagation. Consequently, Model Converges slowly, if the Inputs are not Normalized.

 **Example:** If we perform Image Classification, Size of Image will be very huge, as the Value of each Pixel ranges from 0 to 255. Normalization in this case is very important.



## **Train Test Split**

---



In [120]:

# import packages
import pandas as pd
from keras.utils import np_utils 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE
import numpy as np
np.random.seed(9) # for reproducibility
from keras.models import Sequential

#Train data
X=train.drop('loan_status',1)
y=np_utils.to_categorical(train['loan_status'])

#X_ros,y_ros=ros.fit_resample(X,y)


#Train Test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,stratify=y)

#Scaling of Data
scaler=MinMaxScaler()
X_train_scale=scaler.fit_transform(X_train)
X_test_scale=scaler.transform(X_test)



In [121]:
X_train.shape[1]

49

## **Creating a baseline Neural Network Model**

---

* Here i have created a baseline neural network no sampling or no optimization parameters are tweaked in the code below


In [122]:
#baseline model
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import warnings
warnings.filterwarnings("ignore")
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import numpy as np
np.random.seed(9) # for reproducibility
#Creating a deep learning model
model=Sequential()
# Code starts here
model.add(Dense(49,input_shape=(X_train_scale.shape[1],),activation='relu',kernel_initializer='uniform'))
#model.add(Dense(24,activation='relu',kernel_initializer='uniform'))

#Adding Hidden Layer
model.add(Dense(16,activation='relu',kernel_initializer='uniform'))
model.add(Dense(8,activation='relu',kernel_initializer='uniform'))

#Creating Output Layer
model.add(Dense(10, activation='softmax'))
model.summary()
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 49)                2450      
_________________________________________________________________
dense_59 (Dense)             (None, 16)                800       
_________________________________________________________________
dense_60 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_61 (Dense)             (None, 10)                90        
Total params: 3,476
Trainable params: 3,476
Non-trainable params: 0
_________________________________________________________________


In [123]:
#Fit model on data
model.fit(X_train_scale,y_train, epochs=25, batch_size=100, verbose=2,validation_split=0.2)

Epoch 1/25
3479/3479 - 5s - loss: 0.4250 - accuracy: 0.8901 - val_loss: 0.2247 - val_accuracy: 0.9556
Epoch 2/25
3479/3479 - 5s - loss: 0.2158 - accuracy: 0.9570 - val_loss: 0.2088 - val_accuracy: 0.9595
Epoch 3/25
3479/3479 - 5s - loss: 0.2043 - accuracy: 0.9589 - val_loss: 0.2039 - val_accuracy: 0.9591
Epoch 4/25
3479/3479 - 5s - loss: 0.1902 - accuracy: 0.9596 - val_loss: 0.1799 - val_accuracy: 0.9606
Epoch 5/25
3479/3479 - 5s - loss: 0.1738 - accuracy: 0.9602 - val_loss: 0.1736 - val_accuracy: 0.9588
Epoch 6/25
3479/3479 - 5s - loss: 0.1646 - accuracy: 0.9611 - val_loss: 0.1614 - val_accuracy: 0.9616
Epoch 7/25
3479/3479 - 5s - loss: 0.1606 - accuracy: 0.9617 - val_loss: 0.1595 - val_accuracy: 0.9625
Epoch 8/25
3479/3479 - 5s - loss: 0.1583 - accuracy: 0.9622 - val_loss: 0.1563 - val_accuracy: 0.9625
Epoch 9/25
3479/3479 - 5s - loss: 0.1563 - accuracy: 0.9623 - val_loss: 0.1600 - val_accuracy: 0.9608
Epoch 10/25
3479/3479 - 5s - loss: 0.1551 - accuracy: 0.9622 - val_loss: 0.1532 - 

In [125]:
y_pred=model.predict_classes(X_test_scale)

In [126]:
y_pred=pd.DataFrame(y_pred)

In [127]:
y_pred.columns=['output']
y_pred['output'].value_counts()

1    129002
5     43993
0      9555
7      3446
4       209
9        78
3        67
Name: output, dtype: int64

In [128]:
y_pred

,output
0,1
1,1
2,1
3,5
4,1
...,...
186345,1
186346,1
186347,5
186348,5


In [124]:
#Evaluating Model on Validation
score = model.evaluate(X_test_scale, y_test)
print("Baseline: ")
print(score[1])


5824/5824 [==============================] - 5s 907us/step - loss: 0.1488 - accuracy: 0.9633
Baseline: 
0.9632894992828369


## **Predicting on Test Data**

---



In [129]:
#Selecting features similar to train data
test=test[['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'out_prncp', 'total_pymnt',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'tot_coll_amt',
       'tot_cur_bal', 'emp_length_0', 'emp_length_1', 'emp_length_2',
       'emp_length_3', 'emp_length_4', 'emp_length_5', 'emp_length_7',
       'emp_length_10', 'emp_length_11', 'grade_0', 'grade_1', 'grade_2',
       'grade_3', 'grade_4', 'home_ownership_1', 'home_ownership_4',
       'home_ownership_5', 'initial_list_status_0', 'initial_list_status_1',
       'purpose_1', 'purpose_2', 'purpose_4', 'term_0', 'term_1',
       'verification_status_0', 'verification_status_1',
       'verification_status_2']]

In [130]:
#prediction on test data

#Scaling Test data
test_scale_1=scaler.transform(test)
y_pred_test=model.predict_classes(test_scale_1)
y_pred_test=pd.DataFrame(y_pred_test)
y_pred_test

,0
0,5
1,1
2,1
3,5
4,1
...,...
266209,5
266210,1
266211,1
266212,5


In [131]:
y_pred_test.columns=['loan_status']
y_pred_test['loan_status'].value_counts()
y_pred_test

1    184300
5     62875
0     13606
7      4914
4       286
3       122
9       111
Name: loan_status, dtype: int64

,loan_status
0,5
1,1
2,1
3,5
4,1
...,...
266209,5
266210,1
266211,1
266212,5


## **Optimizing the neural network**

---



In [132]:

# import packages
import pandas as pd
from keras.utils import np_utils 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE
import numpy as np
np.random.seed(9) # for reproducibility
from keras.models import Sequential

from imblearn.over_sampling import RandomOverSampler

#Oversampling of Data
ros=RandomOverSampler(sampling_strategy='minority')
#Train data
X=train.drop('loan_status',1)
y=np_utils.to_categorical(train['loan_status'])

X_ros,y_ros=ros.fit_resample(X,y)


#Train Test split
X_train,X_test,y_train,y_test=train_test_split(X_ros,y_ros,test_size=0.3,stratify=y_ros)

#Scaling of Data
scaler=MinMaxScaler()
X_train_scale=scaler.fit_transform(X_train)
X_test_scale=scaler.transform(X_test)

In [143]:
#Optimizing the Neural Network
from keras import optimizers
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
import warnings
warnings.filterwarnings("ignore")
import numpy as np
np.random.seed(9) # for reproducibility
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization

#Creating a deep learning model
model_2=Sequential()
# Code starts here
model_2.add(Dense(49,input_shape=(X_train_scale.shape[1],),activation='relu',kernel_initializer='uniform'))


#Adding Hidden Layer
model_2.add(Dense(24,activation='relu',kernel_initializer='uniform'))

model_2.add(Dense(16,activation='relu',kernel_initializer='uniform'))

model_2.add(Dense(10, activation='softmax'))

# optimizer 
adam = optimizers.Adam(lr=0.03,beta_1=0.9,beta_2=0.99,decay=0.0015,amsgrad=False)


# early stopping
early_stop = EarlyStopping(monitor='val_loss',restore_best_weights=True)

#Print summary of model
model_2.summary()
# Compile model
model_2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# fit model
model_2.fit(X_train_scale, y_train, batch_size=200, verbose=2, epochs=20,validation_split=0.20, callbacks=[early_stop])

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_74 (Dense)             (None, 49)                2450      
_________________________________________________________________
dense_75 (Dense)             (None, 24)                1200      
_________________________________________________________________
dense_76 (Dense)             (None, 16)                400       
_________________________________________________________________
dense_77 (Dense)             (None, 10)                170       
Total params: 4,220
Trainable params: 4,220
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
2918/2918 - 5s - loss: 0.1610 - accuracy: 0.9604 - val_loss: 0.1106 - val_accuracy: 0.9740
Epoch 2/20
2918/2918 - 5s - loss: 0.1028 - accuracy: 0.9756 - val_loss: 0.1001 - val_accuracy: 0.9765
Epoch 3/20
2918/2918 - 4s - loss: 0.0971 - ac

* ## **We can observe the loss is reduced after optimizing from 0.1488 to 0.0914**

In [144]:
#Predict on Validation Data
y_pred_opt=model_2.predict_classes(X_test_scale)
y_pred_opt=pd.DataFrame(y_pred_opt)
y_pred_opt.columns=['output']
y_pred_opt['output'].value_counts()

1    131220
3    126539
5     43846
0      9322
7      1376
4       261
Name: output, dtype: int64

In [145]:
score = model_2.evaluate(X_test_scale, y_test)
print("optimized Baseline: ")
print(score[1])

9768/9768 [==============================] - 9s 871us/step - loss: 0.0931 - accuracy: 0.9781
optimized Baseline: 
0.9781292676925659


## **Predicting On Test Data**

---



In [146]:
test=test[['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'out_prncp', 'total_pymnt',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'tot_coll_amt',
       'tot_cur_bal', 'emp_length_0', 'emp_length_1', 'emp_length_2',
       'emp_length_3', 'emp_length_4', 'emp_length_5', 'emp_length_7',
       'emp_length_10', 'emp_length_11', 'grade_0', 'grade_1', 'grade_2',
       'grade_3', 'grade_4', 'home_ownership_1', 'home_ownership_4',
       'home_ownership_5', 'initial_list_status_0', 'initial_list_status_1',
       'purpose_1', 'purpose_2', 'purpose_4', 'term_0', 'term_1',
       'verification_status_0', 'verification_status_1',
       'verification_status_2']]

In [147]:
#prediction on test data
test_scale_2=scaler.transform(test)
y_pred_test_2=model_2.predict_classes(test_scale_2)
y_pred_test_2=pd.DataFrame(y_pred_test_2)
y_pred_test_2

,0
0,5
1,1
2,1
3,5
4,1
...,...
266209,5
266210,1
266211,1
266212,5


In [148]:
#Final Predictions
y_pred_test_2.columns=['loan_status']
y_pred_test_2['loan_status'].value_counts()


1    187482
5     62708
0     13297
7      1927
3       426
4       374
Name: loan_status, dtype: int64